# TM10007 Assignment 


In [46]:
# Run this to use from colab environment
!pip install -q --upgrade git+https://github.com/karinvangarderen/tm10007_project.git

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import datasets as ds

## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [48]:
#import the data
from hn.load_data import load_data
data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')

hn_stage = data['label']          # dataframe containing tumor stadium of subject (T12/T34)
data = data.drop(columns='label') # dataframe containing all feature columns except 'label' (tumor stadium of subject)
data.replace(0, np.nan, inplace=True) # replace all zeros with NaN
data = data.dropna(axis='columns')    # dataframe containing all columns without NaN
print(f'The number of columns after preprocessing: {len(data.columns)}')

The number of samples: 113
The number of columns: 160
The number of columns after preprocessing: 141


## Splitting data

Below is a function to split the dataframe into two different dataframes (e.g. the train- and testset). This is done for both the features dataframe as well as the hn_stage dataframe.

In [49]:
# Splitting the data in train- and testpart with a portion of 'test_size' training data (20%)
data_train, data_test, stage_train, stage_test = train_test_split(data, hn_stage, test_size=0.2)

print(f'The number of columns of train data : {len(data_train.index)}')
print(f'The number of columns of train stage : {len(stage_train.index)}')
print(f'The number of columns of test data : {len(data_test.index)}')
print(f'The number of columns of test stage : {len(stage_test.index)}')


The number of columns of train data : 90
The number of columns of train stage : 90
The number of columns of test data : 23
The number of columns of test stage : 23
